In [21]:
import pandas as pd
import numpy as np
import warnings 
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings('ignore')
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
import re


In [10]:
df_pos = pd.read_csv('positive.csv', sep = ';')
df_neg = pd.read_csv('negative.csv', sep = ';')

In [81]:
df = pd.DataFrame(np.vstack((df_pos, df_neg)) )
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11
0,408906692693221377,1386325927,alinakirpicheva,"Да, все-таки он немного похож на него. Но мой ...",1,0,0,0,11825,59,31,2
1,408906695083954177,1386325927,EvgeshaRe,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,1,0,1,0,1273,26,27,0
2,408906695356973056,1386325927,ikonnikova_21,"RT @digger2912: ""Кто то в углу сидит и погибае...",1,0,1,0,1549,19,17,0
3,408906761416867842,1386325943,JumpyAlex,@irina_dyshkant Вот что значит страшилка :D\nН...,1,0,0,0,597,16,23,1
4,408906761769598976,1386325943,JustinB94262583,ну любишь или нет? — Я не знаю кто ты бля:D ht...,1,0,0,0,40,6,16,0


In [16]:
df.shape

(226832, 12)

In [82]:
df = df[3]

In [39]:
morfer = MorphAnalyzer()
sw = get_stop_words("ru")
pattern = r'[^а-яА-Я]'
df = df.apply(lambda x: re.sub(pattern, ' ', x))

def preprocessor_text(line):
    line = line.lower()
    line = " ".join(i for i in line.split() if i not in punctuation).split()
    line = [morfer.parse(i)[0].normal_form for i in line]
    line = [i for i in line if i not in sw and i != "" and i != "—"]
    return line

from string import punctuation
punctuation_update = punctuation

In [41]:

sentences = []
for line in df:
    sentences.append(preprocessor_text(line))
    counter  += 1


In [42]:
!pip install -U gensim

     ---------------------------------------- 24.0/24.0 MB 1.8 MB/s eta 0:00:00
  Attempting uninstall: gensim
    Found existing installation: gensim 3.8.3
    Uninstalling gensim-3.8.3:
      Successfully uninstalled gensim-3.8.3



[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [44]:
from gensim.models import *
from gensim import corpora

In [45]:
dictionary = corpora.Dictionary(sentences)

dictionary.filter_extremes(no_below = 10, no_above = 0.9, keep_n=None) # игнорируем слова, которые встречаются реже 10 раз или составляют более 0.9 словаря 
dictionary.save('tweets.dict')

In [46]:
corpus = [dictionary.doc2bow(text) for text in sentences]
corpora.MmCorpus.serialize('tweets.model', corpus) 

In [61]:
%time lda = ldamodel.LdaModel(corpus, id2word=dictionary, num_topics=8, chunksize=50, update_every=1, passes=2)

Wall time: 2min 3s


In [64]:
lda.show_topics(num_topics=8, num_words=10, formatted=True)

[(0,
  '0.033*"час" + 0.026*"ночь" + 0.022*"скоро" + 0.022*"улица" + 0.021*"холодно" + 0.019*"читать" + 0.018*"заболеть" + 0.018*"фильм" + 0.017*"домой" + 0.014*"твит"'),
 (1,
  '0.024*"утро" + 0.022*"неделя" + 0.017*"написать" + 0.016*"нравиться" + 0.015*"вставать" + 0.014*"казаться" + 0.014*"вроде" + 0.014*"минута" + 0.014*"закончиться" + 0.012*"прийтись"'),
 (2,
  '0.046*"свой" + 0.020*"дело" + 0.017*"учить" + 0.017*"ужасно" + 0.016*"бог" + 0.014*"ранний" + 0.013*"парень" + 0.012*"больно" + 0.011*"температура" + 0.011*"момент"'),
 (3,
  '0.038*"знать" + 0.028*"делать" + 0.028*"любить" + 0.026*"большой" + 0.026*"спать" + 0.024*"писать" + 0.023*"хороший" + 0.022*"болеть" + 0.020*"пойти" + 0.018*"хотеться"'),
 (4,
  '0.031*"сидеть" + 0.026*"ещ" + 0.020*"любимый" + 0.019*"новый" + 0.018*"вчера" + 0.018*"забыть" + 0.017*"остаться" + 0.017*"прийти" + 0.016*"видеть" + 0.013*"работать"'),
 (5,
  '0.055*"завтра" + 0.046*"блин" + 0.044*"вообще" + 0.033*"идти" + 0.031*"школа" + 0.026*"скучать"

In [72]:
import pyLDAvis
import pyLDAvis.gensim as gensimvis

%time vis_data = gensimvis.prepare(lda, corpus, dictionary)
pyLDAvis.display(vis_data)

BrokenProcessPool: A task has failed to un-serialize. Please ensure that the arguments of the function are all picklable.

NameError: name 'vis_data' is not defined

In [73]:
print(lda.log_perplexity(corpus))


-9.391663567431284


In [74]:
print('Персплексия: ', np.exp(lda.log_perplexity(corpus)))

Персплексия:  8.341847025997951e-05


In [80]:
df

0         Да  все таки он немного похож на него  Но мой ...
1                        Ну ты идиотка  я испугалась за ...
2                          Кто то в углу сидит и погибае...
3                         Вот что значит страшилка    Но...
4         ну любишь или нет    Я не знаю кто ты бля     ...
                                ...                        
226827    Но не каждый хочет что то исправлять          ...
226828    скучаю так     только             вправляет мо...
226829            Вот и в школу  в говно это идти уже надо 
226830                             Тауриэль  не грусти  ...
226831    Такси везет меня на работу  Раздумываю приплат...
Name: 3, Length: 226832, dtype: object

In [83]:
coherence_model_lda = CoherenceModel(model=lda, texts=df, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('Средняя когерентность: ', coherence_lda)

TypeError: unsupported operand type(s) for +: 'lil_matrix' and 'lil_matrix'

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [85]:
topics_list = [5, 10, 15]
coherences = []

for num in topics_list:
    lda = ldamodel.LdaModel(corpus, id2word=dictionary, num_topics=num, chunksize=50, update_every=1, passes=2)
    coherences.append(CoherenceModel(model=lda, texts=df, dictionary=dictionary, coherence='c_v').get_coherence())

plt.plot(topics_list, coherences)
plt.xlabel("Число тем")
plt.ylabel("Средняя когерентность")
plt.show()

TypeError: unsupported operand type(s) for +: 'lil_matrix' and 'lil_matrix'

In [77]:
import pyLDAvis.gensim_models
# Визуализация результатов с помощью LDAvis
lda_vis_data = pyLDAvis.gensim_models.prepare(lda, corpus, dictionary)
pyLDAvis.save_html(lda_vis_data, 'lda_visualization.html')

BrokenProcessPool: A task has failed to un-serialize. Please ensure that the arguments of the function are all picklable.